<a href="https://colab.research.google.com/github/Ddkaba/IAD_Lab_1/blob/main/IAD_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.svm import SVC

import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [6]:
dataset = pd.read_csv("https://raw.githubusercontent.com/Ddkaba/IAD_Lab_1/main/V2.csv", index_col=0)
if 'No' in dataset.columns:
    dataset = dataset.drop(columns=['No'])

In [ ]:
# ### BEGIN YOUR CODE
print("Общая информация")
print(dataset.info())

print(f"Количество записей (объектов): {dataset.shape[0]}")
print(f"Количество признаков (фич): {dataset.shape[1]}")

print("\nНазвания столбцов:")
print(dataset.columns.tolist())

print("\nТипы данных:")
print(dataset.dtypes)

print("\nПропущенные значения:")
missing_values = dataset.isnull().sum()
print(missing_values)
print(f"Общее количество пропущенных значений: {missing_values.sum()}")

print("Целевая переменная")
target_column = 'Y house price of unit area'
if target_column in dataset.columns:
    print(f"\nЦелевая переменная: {target_column}")
    print(f"Тип данных целевой переменной: {dataset[target_column].dtype}")
    unique_values = dataset[target_column].unique()
    print(f"Уникальные значения целевой переменной (первые 20): {unique_values[:20]}")
    print(f"Всего уникальных значений: {unique_values.size}")
    if dataset[target_column].nunique() <= 20:
        print("Распределение классов:")
        print(dataset[target_column].value_counts())
        print("Процентное соотношение классов:")
        print(dataset[target_column].value_counts(normalize=True) * 100)

print("Статистика")
print(dataset.describe())

print("Анализ кат. признаков")
categorical_features = []
for col in dataset.columns:
    unique_values = dataset[col].nunique(dropna=True)
    if unique_values <= 10:
        categorical_features.append(col)
        print(f"{col}: {unique_values} уникальных значений - {dataset[col].unique()}")

print(f"\nВсего категориальных признаков: {len(categorical_features)}")

print("Корреляционная матрица (тепловая карта)")

corr_matrix = dataset.corr(numeric_only=True)
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Корреляционная матрица признаков')
plt.tight_layout()
plt.show()

print("Корреляции признаков с целевой переменной:")
if target_column in corr_matrix.columns:
    print(corr_matrix[target_column].sort_values(ascending=False))
# ### END YOUR CODE


Общая информация
<class 'pandas.core.frame.DataFrame'>
Index: 414 entries, 0 to 413
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   X1 transaction date                     414 non-null    float64
 1   X2 house age                            205 non-null    float64
 2   X3 distance to the nearest MRT station  414 non-null    float64
 3   X4 number of convenience stores         414 non-null    int64  
 4   X5 latitude                             414 non-null    float64
 5   X6 longitude                            414 non-null    float64
 6   Y house price of unit area              414 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 25.9 KB
None
Количество записей (объектов): 414
Количество признаков (фич): 7

Названия столбцов:
['X1 transaction date', 'X2 house age', 'X3 distance to the nearest MRT station', 'X4 number of convenience stores', 'X5 l

In [ ]:
# ### BEGIN YOUR CODE
# Удаление ненужных столбцов и разделение на признаки и целевую переменную
target_column = 'Y house price of unit area'
X = dataset.drop([target_column], axis=1)
y = dataset[target_column]

# Корреляция с целевой переменной
dtypes_numeric = ['int64', 'float64']
numerical_data = dataset.select_dtypes(include=dtypes_numeric)
corr_matrix = numerical_data.corr()
print(f"Корреляция с целевой переменной ({target_column}):")
print(corr_matrix[target_column].sort_values(ascending=False))

# Разделение на обучающую и тестовую выборки (80% / 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Числовые признаки (в нашем CSV все признаки числовые)
numerical_features = [
    'X1 transaction date',
    'X2 house age',
    'X3 distance to the nearest MRT station',
    'X4 number of convenience stores',
    'X5 latitude',
    'X6 longitude',
]

# Пайплайн числовых признаков: импутация медианой + стандартизация
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numerical_features),
    ]
)

# Применение пайплайна к данным
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Имена признаков после преобразования
all_feature_names = list(numerical_features)

print(f"Размерность обучающей выборки: {X_train_processed.shape}")
print(f"Размерность тестовой выборки: {X_test_processed.shape}")
print(f"Количество признаков после преобразования: {len(all_feature_names)}")
# ### END YOUR CODE


In [ ]:
# ### BEGIN YOUR CODE
# Гистограммы по всем числовым признакам (без целевой)


# Определим список числовых колонок и исключим целевую, если она есть
feature_columns = dataset.select_dtypes(include=[np.number]).columns.tolist()
if 'Y house price of unit area' in feature_columns:
    feature_columns.remove('Y house price of unit area')

_ = dataset[feature_columns].hist(
    bins=10,
    figsize=(20, 15),
    grid=False,
    edgecolor='black'
)
plt.suptitle('Распределение числовых признаков', y=1.02)
plt.tight_layout()
plt.show()
# ### END YOUR CODE


In [ ]:
# ### BEGIN YOUR CODE
# Геовизуализация: долгота vs широта
import matplotlib.pyplot as plt

lon_col = 'X6 longitude'
lat_col = 'X5 latitude'

# Базовый scatter, как на слайде
ax = dataset.plot(kind='scatter', x=lon_col, y=lat_col, alpha=0.1, figsize=(8, 6))
ax.set_title('Расположение объектов (longitude vs latitude)')
ax.set_xlabel('longitude')
ax.set_ylabel('latitude')
plt.tight_layout()
plt.show()

# Раскраска по целевой (если есть)
if 'Y house price of unit area' in dataset.columns:
    plt.figure(figsize=(9, 7))
    sc = plt.scatter(
        dataset[lon_col], dataset[lat_col],
        c=dataset['Y house price of unit area'], cmap='viridis',
        s=25, alpha=0.6, edgecolors='none'
    )
    cbar = plt.colorbar(sc)
    cbar.set_label('Y house price of unit area')
    plt.title('Геоданные, окрашенные по целевой переменной')
    plt.xlabel('longitude')
    plt.ylabel('latitude')
    plt.tight_layout()
    plt.show()
# ### END YOUR CODE


In [ ]:
# ### BEGIN YOUR CODE
# Эксперименты с комбинациями атрибутов + пересчет корреляций
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

TARGET = 'Y house price of unit area'

dataset_fe = dataset.copy()

# 1) Взаимодействие возраста. Старый и далёкий объект «штрафуется» сильнее, новый рядом со станцией — «премируется». Это даёт модели не одну общую «цену километра», а разную в зависимости от возраста.
age_med = dataset_fe['X2 house age'].median()
age_filled_tmp = dataset_fe['X2 house age'].fillna(age_med)
dataset_fe['age_x_mrt_distance'] = age_filled_tmp * dataset_fe['X3 distance to the nearest MRT station']

# Корреляции
corr = dataset_fe.corr(numeric_only=True)
plt.figure(figsize=(13, 10))
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.title('Корреляции после инженерии признаков')
plt.tight_layout()
plt.show()

if TARGET in corr.columns:
    print('\nТоп-15 корреляций с целевой:')
    print(corr[TARGET].sort_values(ascending=False).head(15))
# ### END YOUR CODE
